# Cake модель

In [1]:

import numpy as np
from itertools import product
from sklearn.linear_model import LinearRegression


import copy


## Общая идея

Линейные модели хорошо предсказывают линейные зависимости, но они не справятся с нелинейной. Однако, любая кривая (или многомерная поверхность) при достаточном приближении также линейна. 

Суть модели в том, что мы "разрежем" наш датасет на кусочки по каждому признаку, которому захотим, т.е. на n-мерные параллепипеды, которые, как кусочки торта скормим уже обычным регрессиям (или любым другим моделям).



In [2]:
class cake:
    
    def __init__(self, chunks = 2, model = LinearRegression(), features = False ):
        # При нулевом и меньшем значении модель ломается, так что делаем защиту от дурака
        if chunks < 1:
            chunks = 1
        #Правильней было бы назвать это "глубина". Chunks - то на сколько кусков модель разрежет КАЖДУЮ фичу 
        #(в итоге мы можем получить огромные значения - общее количество кусков равно параметр Chunk в степени
        #количество признаков)
        self.chunks = chunks 
        #Параметр показывает над какими именно признаками проводить операции. Если указано False (по умолчанию),
        #то над всеми
        self.features = features
        #Признак отвечает за то, какая именно модель будет обучаться на каждом кусочке. Изначально, линейная регрессия,
        #но вы можете поставить любую (например, логистическую)
        self.model = model
    
#Функция создающая фектор. Она принимает вектор (строку нашей матрицы) и возвращает вектор где все "номера кусков"
#на один больше. Номера признаков она не трогает
    def make_second_vector__(self, first_vector):
            
        second_vector = []
        for i in first_vector:
            second_vector.append((i[0], i[1] + 1))
        return second_vector
    

#Вспомогательная функция, которая поможет "разрезать" датасет.    
    def chunking__(self, features, chunks):
        df = self.X
        self.matrix_dict = {}
#Для начала нам надо получить словарь,  в котором будут следующие параметры 
#- номер признака, номер куска и число, определяющее значение признака в данном куске. 
#Запись в словаре вида (0, 1): 5 означает что значение начала второго куска первого признака равна 5 (не забываем, что
#счёт начинается с нуля)
        for i in range(len(self.features_list)):
            for i2 in range(chunks):
                chunk = 0
                #Начало самого первого куска = -бесконечность
                if i2 > 0:
                    chunk = ( 
                    (df[self.features_list[i]].max() - df[self.features_list[i]].min() ) #Длина расброса значений
                    / chunks ) * i2 + df[self.features_list[i]].min()
                else:
                    chunk = -np.inf
                self.matrix_dict[(i,i2)] = chunk 
#Мы заполнили все промежуточные значения, теперь нам надо заполнить последние. Считаем, значения признаков (на тестовой вы-
#борке) могут уходить в бесконечность, потому конец последнего куска будет равен np.inf
        for i in range(len(self.features_list)):
            #chunk = df[self.features_list[i]].max()
            chunk = np.inf
            self.matrix_dict[(i,chunks)] = chunk
        del df #удаляем лишнюю переменную

#Теперь нам понадобится сгенерировать матрицу, которая будет содержать всевозможные пары чисел (номер признака, номер куска)
#Запись вида [(0,0), (1,0), ...] будет означать "первый признак, первый кусок, второй признак и первый кусок и т.п."
        self.matrix = []
        matrix = []
        g = list(range(chunks))
        
        matrix = list(product( g, repeat = len(self.features_list)) )
        for i in matrix:
            vector = []
            for i2 in range(len(i)):
                vector.append((i2, i[i2]))
            self.matrix.append(vector)
        
        
#Функция обучения модели. Кто бы мог подумать?        
    def  fit(self, X,y):
        #ATTENTION! АХТУНГ! ВНИМАНИЕ! Если соберётесь делать что-то подобное, используйте copy()! 
        #До этого я много часов потратил, чтобы понять почему модель работает из рук вон плохо, а при попытке засунуть в неё
        #логистическую регрессию - выдаёт исключения и не хочет работать! Подробней будет описано ниже.
        self.X = X.copy()
        self.X = self.X.reset_index(drop=True)
        self.y = y.copy()
        self.y = self.y.reset_index(drop=True)
        
        
        self.model_matrix = {}
        #Настраиваем признаки по которым будем делить датасет
        if self.features == False:
            self.features_list = list(X.columns)
        else:
            self.features_list = self.features
        #Используем функцию, которая разбивает датасет по кусочком, а точнее, создаёт индексы всех кучочков и словарь,
        #в котором содержатся численные выражения этих самых индексов
        self.chunking__( self.features_list, self.chunks)
        
        #Что если у нас не окажется данных в каких-то кусках, но они будут в тестовой выборке? Тогда мы не сможем исполь-
        #зовать наши модели, и будем делать предсказания из общих оснований. Т.е. обучим модель на ВСЕЙ выборке и будем
        #использовать ЕЁ предсказания
        self.base_model = copy.copy(self.model)
        self.base_model.fit(X,y)
        
        #Список для наших моделей
        models = []
        

        #Теперь нам нужно обучить наши модели на кусочках нашего датасета. Для этого нам нужно извлечь сами кусочки.
        for i in self.matrix:
        #е - второй вектор, где все значения индекса куска больше на единицу
            
            e = self.make_second_vector__(i)
            
            values = []
            for p in range(len(i)):
                a = ''
                if p < (len(i) - 1 ):
                    a = ' and '
                #В начале должен быть знак строго меньше, а в конце <=, иначе могут возникнуть пересечения кусков, что 
                #поломает всю модель
                values.append(str(self.matrix_dict[i[p]]) + '<' + 
                              self.features_list[i[p][0]] + '<=' + str(self.matrix_dict[e[p]]) + a)
            #цикл выше генерирует строку, которую мы пихаем в df.query чтобы получить нужный нам кусочек, а точнее его 
            #индексы
            values = ''.join(values)
            indexes = self.X.query(values).index
            #Теперь, наконец, создаём тренировочную мини-выборку из одного кусочка для наших моделей. 
            X_train = self.X.iloc[indexes]
            y_train = self.y.iloc[indexes]
            
            if not X_train.empty: #Нам вполне могут попасться кусочки без данных, это нужно учесть.
                #Если так случилось, то вместо модели запоминаем пустое значение
                
                #Некоторые модели чувствительны к составу данных в целевом признаке, например, логистическая регрессия 
                #выдаст исключение, если там только нули или только единицы. А такое вполне возможно, так что используем
                #try-except
                try:
                    this_model = self.model.fit(X_train, y_train)    
                except:
                    this_model = None
            else: 
                this_model = None
            model_matrix_append = { (tuple(i),tuple(e)) : copy.copy(this_model)}
            #Тут как видите, я засовываю в матрицу не просто this_model, а copy.copy(this_model). Если так не делать, то
            #определяя this_model заново, я буду перезаписывать все модели в матрице. Т.е. по задумке у меня на каждый
            #кусочек данных своя модель, отвечающая именно за него, а до этого во всех кусках была ПОСЛЕДНЯЯ модель.
            #Логично, что датасет работал хуже обычной регрессии, так как по сути использовал ту же регрессию, но обученную
            #на малом куске данных!
            del this_model
            self.model_matrix.update(model_matrix_append)
        
        
    
    def predict(self, X):
    #Теперь, когда мы обучили наши модели, нужно предсказывать. Так же как и в обучении, мы пройдёмся по всей матрице и
    #индексами начал кусков и разобьём тестовый датасет на эти самые куски. Затем мы будем брать модель, обученную на 
    #аналогичных кусках и с помощью неё делать предсказание (только для этого куска)
        
        self.Xtest = X.copy().reset_index(drop=True)
        #Если не дропать индексы, получится каша, предсказания будут перепутаны и всё такое.
        self.predictions = []
        predictions = []

        for i in self.matrix: #Да-да, я копирую большую часть кода. Индусы могли бы мной гордиться.
        #е - второй вектор, где все значения индекса куска больше на единицу. Как и в предыдущем коде.
            
            e = self.make_second_vector__(i)
            
            values = []
            for p in range(len(i)):
                a = ''
                if p < (len(i) - 1 ):
                    a = ' and '
                values.append(str(self.matrix_dict[i[p]]) + '<' + 
                              self.features_list[i[p][0]] + '<=' + str(self.matrix_dict[e[p]]) + a)
            #цикл выше генерирует строку, которую мы пихаем в df.query чтобы получить нужный нам кусочек, а точнее его 
            #индексы
            values = ''.join(values)
            
            indexes = self.Xtest.query(values).index                
            #Создаём тот кусочек данных, на которых мы будем предсказывать. 
            X_test = self.Xtest.iloc[indexes]
            X_test = X_test.reset_index(drop=True)
            
            if not X_test.empty: #Нам вполне могут попасться кусочки без данных, это нужно учесть.

                #Выбираем модель из матрицы
                this_model = self.model_matrix[(tuple(i),tuple(e))]
                
                #Если модели нет, то используем обученную на всех данных, если есть, то из матрицы
                if this_model != None:
                    
                    local_predictions = this_model.predict(X_test)

                else:
                    
                    local_predictions = self.base_model.predict(X_test)

            #Теперь создаём список из кортежей, где первый элемент кортежа инлдекс, а второй - предсказание. Добавляем
            #к глобальному списку предсказаний
                predictions += list(zip(indexes, local_predictions))
                
        #Выходим из цикла
        
        #Сортируем предсказания по индексу (т.е. первому элементу кортежа)
        predictions.sort(key = lambda x: x[0])
        
        #превращаем наш список кортежей в нормальный список, который и будем возвращать.
        for i in predictions:
            self.predictions.append(i[1])
            
        self.predictions = np.array(self.predictions)
        return self.predictions